In [2]:
from urllib.error import HTTPError
import requests
import pandas as pd

In [3]:
location = '30.000 N 90.000 W (30&#176;0\'0" N 90&#176;0\'0" W)'
location = "55.000 S 90.000 W (55&#176;0\'0\" S 90&#176;0\'0\" W)	"
def lat_lng(location):
    try:
        tokens = location.split()
        lat = 1 if tokens[1] == 'N' else -1
        lng = 1 if tokens[3] == 'E' else -1

        return ( lat*float(tokens[0]), lng*float(tokens[2]) )
    except:
        return None

lat_lng(location)

(-55.0, -90.0)

In [9]:
stations_file = 'https://www.ndbc.noaa.gov/data/stations/station_table.txt'
raw_output = 'raw/station_table.txt'

def save_file(source_url, target_file_path):
    response = requests.get(source_url)
    with open (target_file_path, "w") as f:
        f.write(response.text)

save_file(stations_file, raw_output)

In [5]:
stations = pd.read_csv(stations_file,sep="|")
stations.columns = [ c.replace("#","").strip() for c in stations.columns]
stations['LATLNG'] = stations.apply(lambda r: lat_lng(r['LOCATION']), axis=1)
mb_stations = stations[ stations['TTYPE'].str.find('Moored') > -1 ]
readings = pd.DataFrame()
mb_stations

,STATION_ID,OWNER,TTYPE,HULL,NAME,PAYLOAD,LOCATION,TIMEZONE,FORECAST,NOTE,LATLNG
148,34002,OO,Moored Buoy,NaN,OOI Southern Ocean Buoy,NaN,"55.000 S 90.000 W (55&#176;0'0"" S 90&#176;0'0"" W)",C,,NaN,"(-55.0, -90.0)"
170,41024,CR,Moored Buoy,NaN,"Sunset Nearshore, NC (SUN2)",NaN,"33.837 N 78.477 W (33&#176;50'14"" N 78&#176;28...",E,FZUS52.KILM,Right whales are active off NC from November t...,"(33.837, -78.477)"
174,41029,CR,Moored Buoy,NaN,"Capers Nearshore, SC (CAP2)",NaN,"32.803 N 79.624 W (32&#176;48'12"" N 79&#176;37...",E,FZUS52.KCHS,Right whales are active off SC from November t...,"(32.803, -79.624)"
176,41033,CR,Moored Buoy,NaN,"Fripp Nearshore, SC (FRP2)",NaN,"32.279 N 80.406 W (32&#176;16'44"" N 80&#176;24...",E,FZUS52.KCHS,Right whales are active off SC from November t...,"(32.279, -80.406)"
179,41037,CR,Moored Buoy,NaN,"Wrightsville Beach Offshore, NC (ILM3)",NaN,"33.988 N 77.362 W (33&#176;59'16"" N 77&#176;21...",E,FZUS52.KILM FZNT22.KWBC,Right whales are active off NC from November t...,"(33.988, -77.362)"
180,41038,CR,Moored Buoy,NaN,"Wrightsville Beach Nearshore, NC (ILM2)",NaN,"34.141 N 77.715 W (34&#176;8'27"" N 77&#176;42'...",E,FZUS52.KILM,Right whales are active off NC from November t...,"(34.141, -77.715)"
189,41051,CI,Moored Buoy,NaN,"South of St. Thomas, VI",NaN,"18.257 N 65.004 W (18&#176;15'26"" N 65&#176;0'...",?,FZCA52.TJSJ,NaN,"(18.257, -65.004)"
190,41052,CI,Moored Buoy,NaN,"South of St. John, VI",NaN,"18.249 N 64.763 W (18&#176;14'56"" N 64&#176;45...",N,FZCA52.TJSJ,NaN,"(18.249, -64.763)"
191,41053,CI,Moored Buoy,NaN,"San Juan, PR",NaN,"18.474 N 66.099 W (18&#176;28'27"" N 66&#176;5'...",N,FZCA52.TJSJ,NaN,"(18.474, -66.099)"
192,41056,CI,Moored Buoy,NaN,"Vieques Island, PR",NaN,"18.261 N 65.464 W (18&#176;15'40"" N 65&#176;27...",N,FZCA52.TJSJ,NaN,"(18.261, -65.464)"


In [11]:
print("Gathering Realtime Weather Data for Moored Bouys from noaa.gov")
for row in mb_stations.to_records()[:5]:
    try:
        file = f'https://www.ndbc.noaa.gov/data/realtime2/{row["STATION_ID"]}.txt'
        file_out = f'raw/{row["STATION_ID"]}.txt'
        df=pd.read_csv(file, delim_whitespace=True)
        df['READING_STATION_ID'] = row["STATION_ID"]
        save_file(file, file_out)        
        print(f"[  OK   ] {file}")    
        readings = readings.append( df.loc[1:])
    except HTTPError:
        print(f"[MISSING] {file}")        
        pass
    
readings

Gathering Realtime Weather Data for Moored Bouys from noaa.gov
[MISSING] https://www.ndbc.noaa.gov/data/realtime2/34002.txt
[  OK   ] https://www.ndbc.noaa.gov/data/realtime2/41024.txt
[  OK   ] https://www.ndbc.noaa.gov/data/realtime2/41029.txt
[  OK   ] https://www.ndbc.noaa.gov/data/realtime2/41033.txt
[  OK   ] https://www.ndbc.noaa.gov/data/realtime2/41037.txt


,#YY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,PTDY,TIDE,READING_STATION_ID
1,2021,09,11,14,08,50,4.0,5.0,MM,MM,MM,MM,1025.3,24.5,27.1,MM,MM,MM,MM,41024
2,2021,09,11,13,08,60,4.0,4.0,MM,MM,MM,MM,1024.9,23.5,27.1,MM,MM,MM,MM,41024
3,2021,09,11,12,08,50,4.0,6.0,MM,MM,MM,MM,1024.8,22.6,27.2,MM,MM,MM,MM,41024
4,2021,09,11,11,08,50,4.0,6.0,MM,MM,MM,MM,1023.7,22.8,27.2,MM,MM,MM,MM,41024
5,2021,09,11,10,08,60,4.0,6.0,MM,MM,MM,MM,1022.8,23.1,27.2,MM,MM,MM,MM,41024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086,2021,07,28,04,08,180,6.0,8.0,MM,MM,MM,MM,1015.2,28.0,27.8,MM,MM,MM,MM,41037
1087,2021,07,28,03,08,190,5.0,7.0,MM,MM,MM,MM,1015.3,27.9,27.8,MM,MM,MM,MM,41037
1088,2021,07,28,02,08,190,4.0,6.0,MM,MM,MM,MM,1015.5,28.1,27.9,MM,MM,MM,MM,41037
1089,2021,07,28,01,08,190,5.0,7.0,MM,MM,MM,MM,1015.4,28.2,27.9,MM,MM,MM,MM,41037


In [ ]:
combined = mb_stations.merge(readings, how='inner', left_on='STATION_ID',right_on='READING_STATION_ID' )
combined

,STATION_ID,OWNER,TTYPE,HULL,NAME,PAYLOAD,LOCATION,TIMEZONE,FORECAST,NOTE,...,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,PTDY,TIDE,READING_STATION_ID
0,41024,CR,Moored Buoy,NaN,"Sunset Nearshore, NC (SUN2)",NaN,"33.837 N 78.477 W (33&#176;50'14"" N 78&#176;28...",E,FZUS52.KILM,Right whales are active off NC from November t...,...,MM,MM,1025.3,24.5,27.1,MM,MM,MM,MM,41024
1,41024,CR,Moored Buoy,NaN,"Sunset Nearshore, NC (SUN2)",NaN,"33.837 N 78.477 W (33&#176;50'14"" N 78&#176;28...",E,FZUS52.KILM,Right whales are active off NC from November t...,...,MM,MM,1024.9,23.5,27.1,MM,MM,MM,MM,41024
2,41024,CR,Moored Buoy,NaN,"Sunset Nearshore, NC (SUN2)",NaN,"33.837 N 78.477 W (33&#176;50'14"" N 78&#176;28...",E,FZUS52.KILM,Right whales are active off NC from November t...,...,MM,MM,1024.8,22.6,27.2,MM,MM,MM,MM,41024
3,41024,CR,Moored Buoy,NaN,"Sunset Nearshore, NC (SUN2)",NaN,"33.837 N 78.477 W (33&#176;50'14"" N 78&#176;28...",E,FZUS52.KILM,Right whales are active off NC from November t...,...,MM,MM,1023.7,22.8,27.2,MM,MM,MM,MM,41024
4,41024,CR,Moored Buoy,NaN,"Sunset Nearshore, NC (SUN2)",NaN,"33.837 N 78.477 W (33&#176;50'14"" N 78&#176;28...",E,FZUS52.KILM,Right whales are active off NC from November t...,...,MM,MM,1022.8,23.1,27.2,MM,MM,MM,MM,41024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4213,41037,CR,Moored Buoy,NaN,"Wrightsville Beach Offshore, NC (ILM3)",NaN,"33.988 N 77.362 W (33&#176;59'16"" N 77&#176;21...",E,FZUS52.KILM FZNT22.KWBC,Right whales are active off NC from November t...,...,MM,MM,1015.2,28.0,27.8,MM,MM,MM,MM,41037
4214,41037,CR,Moored Buoy,NaN,"Wrightsville Beach Offshore, NC (ILM3)",NaN,"33.988 N 77.362 W (33&#176;59'16"" N 77&#176;21...",E,FZUS52.KILM FZNT22.KWBC,Right whales are active off NC from November t...,...,MM,MM,1015.3,27.9,27.8,MM,MM,MM,MM,41037
4215,41037,CR,Moored Buoy,NaN,"Wrightsville Beach Offshore, NC (ILM3)",NaN,"33.988 N 77.362 W (33&#176;59'16"" N 77&#176;21...",E,FZUS52.KILM FZNT22.KWBC,Right whales are active off NC from November t...,...,MM,MM,1015.5,28.1,27.9,MM,MM,MM,MM,41037
4216,41037,CR,Moored Buoy,NaN,"Wrightsville Beach Offshore, NC (ILM3)",NaN,"33.988 N 77.362 W (33&#176;59'16"" N 77&#176;21...",E,FZUS52.KILM FZNT22.KWBC,Right whales are active off NC from November t...,...,MM,MM,1015.4,28.2,27.9,MM,MM,MM,MM,41037


In [ ]:
stations[' LOCATION '][1]

'30.000 N 90.000 W (30&#176;0\'0" N 90&#176;0\'0" W)'

(30.0, -90.0)

148          (55.0, -90.0)
170      (33.837, -78.477)
174      (32.803, -79.624)
176      (32.279, -80.406)
179      (33.988, -77.362)
180      (34.141, -77.715)
189      (18.257, -65.004)
190      (18.249, -64.763)
191      (18.474, -66.099)
192      (18.261, -65.464)
193      (19.833, -70.731)
194      (18.476, -65.157)
199      (34.207, -76.949)
269       (30.09, -88.212)
301      (17.869, -66.532)
302      (11.185, -60.848)
304      (19.699, -80.061)
305       (18.432, -69.58)
343      (43.762, -69.988)
344      (40.883, -73.728)
346      (42.325, -65.909)
351      (42.523, -70.566)
352      (43.179, -70.426)
354      (43.715, -69.355)
355      (44.055, -68.996)
356      (44.103, -68.112)
359      (43.497, -67.876)
361      (41.138, -72.655)
362       (40.956, -73.58)
363      (37.204, -76.777)
364      (38.033, -76.335)
365      (39.152, -76.391)
366      (38.389, -76.526)
369      (38.883, -75.183)
370      (39.122, -75.256)
372       (39.54, -76.074)
373      (37.567, -76.257)
3